TESTS ON THE SOLAR WIND PART OF THE CODE.

In [1]:
import os
import scipy.io as sio
from scipy.io import readsav
from scipy.optimize import curve_fit
from scipy import optimize
import numpy as np
from math import *
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib qt 

from star import Star 
from planet import Planet 
from dynamo_region import DynamoRegion
from magnetic_moment import MagneticMoment
from stellar_wind import StellarWind
from target_selection import Config

In [2]:
config_param = Config()

Constants for Jupiter and the Sun

In [3]:
MS=1.989e30 #kg
RS=6.96342e8 #m
AS=4.6e9 #yo
BS= 1.435e-4 #T
LS=3.826e26 #W

MJ=1.8986e27 #kg
#RJ=69911e3   #m
RJ=71492e3
wJ=1.77e-4   #s-1
LJ_norm=1.31e-9
dJ=5.2*149597870700

sun = Star(
    name="Sun",
    mass=1.0,
    radius={"models": config_param.star_radius_models, "radius": 1.0},
    age=AS,
    obs_dist=1.0,
    sp_type ='V',
    )
jup = Planet(
    name="Jupiter",
    mass=1.0,
    radius={"models": config_param.planet_radius_models, "radius": 1.0},
    distance=5.2,
    worb={"star_mass": MS, "worb": 1.0},
    luminosity={
        "models": config_param.planet_luminosity_models,
        "luminosity": np.nan,
        "star_age": 4.6,
    },
    wrot=1.0,
)

In [4]:
jup.tidal_locking(age=4.6e9, star_mass=1.0)
sun.compute_effective_temperature(np.nan)

In [5]:
dyn_region_jup = DynamoRegion.from_planet(planet=jup, rhocrit=config_param.rho_crit)
dyn_region_jup.magnetic_field(planet=jup,rc_dyn=config_param.rc_dyn, jup=True)
mag_moment_jup = MagneticMoment(models=config_param.magnetic_moment_models, Mm=1.56e27, Rs=1.0)
vjup, vejup, nejup, Tjup = StellarWind._Parker(star=sun, planet=jup, T=0.81e6)
print(vjup)
sun.compute_magnetic_field(value= {'model': config_param.star_magfield_models, 'mag_field' : 1.435})

planet dist > 1UA
522686.64156545105


In [24]:
sun.magfield

0.00014350000000000002

In [6]:
G = 6.6725985e-11  # N.m^2/kg^2
dua = 1.49597870700e11  # m
Psun = 25.5  # days
vorb = sqrt(
    G * MS / (5.2 * dua)
)
beta = atan(vorb / vjup)

Bimf_r, Bimf_p = StellarWind._calc_Bimf(stardist=5.2,magfield_surf = BS)
Bimf_r0, Bimf_p0 = StellarWind._calc_Bimf(stardist=5.2,magfield_surf = None)
print(Bimf_r,Bimf_p)
print(Bimf_r0,Bimf_p0)

alpha = atan(Bimf_p / Bimf_r)
alpha0 = atan(Bimf_p0 / Bimf_r0)

Bsw = sqrt(Bimf_r**2 + Bimf_p**2) * abs(sin(alpha - beta))
Bsw0 = sqrt(Bimf_r0**2 + Bimf_p0**2) * abs(sin(alpha0 - beta))
print(Bsw,Bsw0)

TypeError: _calc_Bimf() missing 3 required positional arguments: 'R_star', 'vsw', and 'rotperiod'

In order to estimate the power of the emissions, we need to estimate the velocity and the density of stellar wind at the planet, then derive the intensity of the magnetic field carried by the SW.

We test the Parker model for the solar wind velocity for several type of stars with the following parameters : 
- G-type star : M=Msun, R=Rsun ;
- K/M-type star : M=0.5*Msun, R=0.46*Rsun

We computed the values of the velocity and the density for several distances and ages of the stars always adjusting the corona temperature such as that 1UA, v=425km/s.
We find Tcorona=0.81e6 K for G-type stars and Tcorona= for K/M-type stars.

- G-type star, 
- - t=4.6 Gyr, Tcorona = 0.81e6 K
- - t=1 Gyr, Tcorona= 2.09e6 K
- - t=0.7 Gyr, Tcorona= 2.61e6 K

In [ ]:
(1e6*4.6e9)/(4.6e9+1e9)

In [7]:
vo=3397e3 #m/s 
no=1.6e10 #m-3
dua=1.49597870700e11 #m
G=6.6725985e-11 #N.m^2/kg^2


test_planet = Planet(
    name='planet_test',
    mass=1.0,
    radius={"models": config_param.planet_radius_models, "radius": 1.0},
    distance=0.05,
    worb={"star_mass": 1.0, "worb": 1.0},
    luminosity={
        "models": config_param.planet_luminosity_models,
        "luminosity": np.nan,
        "star_age": 4.6,
    },
    detection_method='VR',
    wrot=1.0,
)
star_test = Star(
    name='star_test',
    mass=1.0,
    radius={"models": config_param.star_radius_models, "radius": 1.0},
    age=4.6,
    obs_dist=1.0,
    sp_type = str('V'),
    )


v,veff,n,Tcor=StellarWind._Parker(star_test,test_planet)
vorb=sqrt(G*star_test.mass/(0.05*dua))
print(v/1e3, veff/1e3, vorb/1e3, Tcor/1e6)

164.19481567631007 211.43139798013246 9.444952726739055e-14 0.8108571428571428


- - v(d), n(d) à t=4.6 Gyr

In [8]:
list_dist=np.linspace(0.01,1,100)
list_age = np.array([0.7,1,3,6])
vd=np.zeros(shape=(4,100)) ; nd=np.zeros(shape=(4,100)) 
veffd=np.zeros(shape=(4,100)) ; vorb=np.zeros(shape=(4,100)) ; Tcor=np.zeros(shape=(4,100))

for k in range(4):
    for i in range(0,len(list_dist)):
        test_planet = Planet(
            name='planet_test',
            mass=1.0,
            radius={"models": config_param.planet_radius_models, "radius": 1.0},
            distance=list_dist[i],
            worb={"star_mass": 1.0, "worb": 1.0},
            luminosity={
                "models": config_param.planet_luminosity_models,
                "luminosity": np.nan,
                "star_age": list_age[k],
            },
            detection_method='VR',
            wrot=1.0,
        )

        star_test = Star(
            name='HD 189733',
            mass=1.0,
            radius={"models": config_param.star_radius_models, "radius": 1.0},
            age=list_age[k],
            obs_dist=1.0,
            sp_type = str('V'),
            )
        
        vd[k,i],veffd[k,i],nd[k,i],Tcor[k,i]=StellarWind._Parker(star_test,test_planet)
        vorb[k,i]=sqrt(G*star_test.unnormalize_mass()/(list_dist[i]*dua))




planet dist > 1UA
planet dist > 1UA
planet dist > 1UA
planet dist > 1UA


In [10]:
plt.figure(figsize=(10,7))
#plt.plot(list_age,vd/1e3, label='v')
for k in range(4):
    plt.plot(list_dist,veffd[k,:]/1e3,label='$t_*$ = {} Gyrs'.format(list_age[k]))
#plt.plot(list_age, vorb/1e3, label='vorb')
#plt.title("v(d)")
plt.xlabel('$d_{*-p}$ [UA]', fontsize=18)
plt.ylabel("$v_{eff}$ [km/s]", fontsize =18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_veff_dist.png', transparent=True, bbox_inches='tight' ,dpi=150 )
plt.show()

In [11]:
plt.figure(figsize=(10,7))
#plt.plot(list_age,vd/1e3, label='v')
for k in range(4):
    plt.plot(list_dist,nd[k,:]/1e3,label='$t_*$ = {} Gyrs'.format(list_age[k]))
#plt.plot(list_age, vorb/1e3, label='vorb')
#plt.title("v(d)")
plt.xlabel('$d_{*-p}$ [UA]', fontsize=18)
plt.ylabel("$n_e$ [m$^{-3}$]", fontsize =18)
plt.xscale('log')
plt.yscale('log')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_ne_dist.png', transparent=True, bbox_inches='tight',dpi=150 )
plt.show()


- - v(t) and n(t) at 1 UA

In [ ]:
list_time=np.linspace(0.7, 6, 100)*1e9
vt=np.zeros(100) ; nt=np.zeros(100)
tau=2.56e7 #yr

for i in range(0,100):
    vt[i]=vo/pow((1+(list_time[i]/tau)),0.4)
    nt[i]=no/pow((1+(list_time[i]/tau)),1.5)

plt.figure()
plt.plot(list_time/1e9,vt/1e3)
plt.xlabel('t [Gyr]')
plt.ylabel( 'v[km/s]')
plt.title('v(t)')
plt.grid()


In [ ]:
plt.figure()
plt.plot(list_time/1e9,nt)
plt.xlabel('t [Gyr]')
plt.ylabel( ' n[/m3')
plt.title('n(t)')
plt.yscale('log')
plt.grid()

In [ ]:
print(vt[6],vt[0],vt[73])
print(nt[6],nt[0],nt[73])

- K/M-type stars : 
- - t=4.6 Gyr, Tcorona=0.695e6 K
- - t=1 Gyr, Tcorona=1.84e6 K
- - t=0.7 Gyr, Tcorona=2.31e6 K

In [ ]:
vo=3397e3 #m/s 
no=1.6e10 #m-3
dua=1.49597870700e11 #m

test_planet=Planet('planet_test',1.0,1.0,1.0,1.0,1.0,0.05*dua)
star_test=Star('star_test', 0.5*1.989e30,0.46,1e9,1.0,1.0)
v,veff,n,Tcor=parker(test_planet,star_test)

print(v,veff,n)


- Test of the function computing the appropriate Tcorona for any case. We adjust the temperature by steps of 0.005e6 K until the velocity is in the range of vsun +/- 2 km/s. Then we adjust the temperature more precisely by steps of 0.001e6 K until the velocity is in the range Vsun +/- 0.5 km/s. We note that it takes about 50s to compute 10000 values. 

In [ ]:
list_time=np.linspace(0.7, 6, 100)*1e9
list_mass=np.linspace(0.1,2,100)*1.989e30
T=np.zeros(shape=[100,100])

for i in range(0,100):
    for j in range(0,100):
        T[j][i]=calc_temperature(list_mass[j],list_time[i])

In [ ]:
plt.figure()
ax1=plt.gca()
im1=ax1.imshow(T,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_mass[0]/1.989e30,list_mass[99]/1.989e30],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('Mass [M$_{sol}$]')
plt.title('T$_{corona}$ as a function of stellar age and mass')
cb1=plt.colorbar(im1, ax=ax1)
plt.tight_layout()
plt.show()

- Test of the Parker model for the velocity.

- - As a function of stellar age and mass, R=1Rsol and d=1UA.

In [ ]:
list_time=np.linspace(0.7, 6, 100)*1e9
list_mass=np.linspace(0.1,2,100)*1.989e30
v1=np.zeros(shape=[100,100])
veff1=np.zeros(shape=[100,100])
n1=np.zeros(shape=[100,100])
T1=np.zeros(shape=[100,100])

for i in range(0,100):
    for j in range(0,100):
        planet1=Planet('pla_test',1.0,1.0,1.0,1.0,1.0,dua)
        star1=Star('star_test',list_mass[j],1.0,list_time[i],1.0,1.0)
        v1[j][i],veff1[j][i],n1[j][i],T1[j][i]=parker(planet1,star1)

In [ ]:
plt.figure()
ax1=plt.gca()
im1=ax1.imshow(v1/1e3,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_mass[0]/1.989e30,list_mass[99]/1.989e30],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('Mass [M$_{sol}$]')
plt.title('V as a function of stellar age and mass')
cb1=plt.colorbar(im1, ax=ax1)
cb1.set_label('km.s$^{-1}$')
plt.tight_layout()
plt.show()

plt.figure()
ax2=plt.gca()
im2=ax2.imshow(veff1/1e3,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_mass[0]/1.989e30,list_mass[99]/1.989e30],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('Mass [M$_{sol}$]')
plt.title('V$_{eff}$ as a function of stellar age and mass')
cb2=plt.colorbar(im2, ax=ax2)
cb2.set_label('km.s$^{-1}$')
plt.tight_layout()
plt.show()

plt.figure()
ax3=plt.gca()
im3=ax3.imshow(n1,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_mass[0]/1.989e30,list_mass[99]/1.989e30],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('Mass [M$_{sol}$]')
plt.title('n as a function of stellar age and mass')
cb3=plt.colorbar(im3, ax=ax3)
cb3.set_label('m$^{-3}$')
plt.tight_layout()
plt.show()

- - As a function of stellar age and star-planet distance, R=1Rsol, M=1Msol

In [ ]:
list_time=np.linspace(0.7, 6, 100)*1e9
list_dist=np.linspace(0.01,1,100)
v2=np.zeros(shape=[100,100])
veff2=np.zeros(shape=[100,100])
n2=np.zeros(shape=[100,100])
T2=np.zeros(shape=[100,100])

for i in range(0,100):
    for j in range(0,100):
        planet1=Planet('pla_test',1.0,1.0,1.0,1.0,1.0,list_dist[j]*dua)
        star1=Star('star_test',1.989e30,1.0,list_time[i],1.0,1.0)
        v2[j][i],veff2[j][i],n2[j][i],T2[j][i]=parker(planet1,star1)

In [ ]:
plt.figure()
ax1=plt.gca()
im1=ax1.imshow(v2/1e3,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_dist[0],list_dist[99]],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('d$_{*-p}$ [AU]')
plt.title('V as a function of stellar age and d$_{*-p}$')
cb1=plt.colorbar(im1, ax=ax1)
cb1.set_label('km.s$^{-1}$')
plt.tight_layout()
plt.show()

plt.figure()
ax2=plt.gca()
im2=ax2.imshow(veff2/1e3,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_dist[0],list_dist[99]],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('d$_{*-p}$ [AU]')
plt.title('V$_{eff}$ as a function of stellar age and d$_{*-p}$')
cb2=plt.colorbar(im2, ax=ax2)
cb2.set_label('km.s$^{-1}$')
plt.tight_layout()
plt.show()

plt.figure()
ax3=plt.gca()
im3=ax3.imshow(n2,origin='lower',aspect='auto', extent=[list_time[0]/1e9,list_time[99]/1e9,list_dist[0],list_dist[99]],cmap='viridis')
plt.xlabel('Age [Gyr]')
plt.ylabel('d$_{*-p}$ [AU]')
plt.title('n as a function of stellar age and d$_{*-p}$')
cb3=plt.colorbar(im3, ax=ax3)
cb3.set_label('m$^{-3}$')
plt.tight_layout()
plt.show()

- Taking into account the coronal mass ejection (CME) contribution to the stelar wind.
We can expect CME collisions for close-in exoplanets. See Khodachenko et al,2006 :  they made an estimation of the dependence of the average n_cme and v_cme with respect to the substellar distance. They also show that individual CME's have very different speed but the average CME velocity is almost independant of the substellar distance : v_cme=5.26e5 m/s.


In [ ]:
vo=3397e3 #m/s 
no=1.6e10 #m-3
dua=1.49597870700e11 #m
G=6.6725985e-11 #N.m^2/kg^2

test_planet=Planet('planet_test',1.0,1.0,1.0,1.0,1.0,1*dua)
star_test=Star('star_test', 1.989e30,1.0,4.6e9,1.0,1.0)
v,veff,n,Tcor=parker(test_planet,star_test)
vcme,veff_cme,ns_cme, T_cme=CME(star_test,test_planet)
vorb=sqrt(G*star_test.mass/(0.2*dua))
print(v/1e3, veff/1e3, n,Tcor/1e6)
print(vcme/1e3, veff_cme/1e3, ns_cme, T_cme/1e6)

    Test nouvelle méthode calcul de v_sw

In [ ]:
dua = 1.49597870700e11  # m
kb = 1.380658e-23  # J/K
mp = 1.660540210e-27  # kg
G = 6.6725985e-11  # N.m^2/kg^2
M = 1.989e30  # kg
T=812428.5714285715 # K
d = 0.03 # UA
vc = sqrt(2 * kb * T / mp)
rc = mp * G * M / (4 * kb * T)
vorb = sqrt(G * M / (d * dua))

d_temp=1.0 
v_temp_ini = optimize.newton(
    parker_velocity, 350.0e3,
    parker_velocity_p, args=(d_temp * dua,rc,vc),
    maxiter=50,
    )
print(optimize.newton(
    parker_velocity, 50.0e3,
    parker_velocity_p, args=(d * dua,rc,vc),
    maxiter=50,
    ))
while ((abs(d_temp - d ) >= 1e-5)):
    d_temp = (9*d_temp + d)/10.0
    v_temp = optimize.newton(
        parker_velocity, v_temp_ini,
        parker_velocity_p, args=(d_temp * dua,rc,vc),
        maxiter=50,
        )
    if (v_temp/vc > 1.0) and (d_temp/(rc/dua) <= 1.) :
        v_temp = optimize.newton(
        parker_velocity, 0.9*v_temp_ini,
        parker_velocity_p, args=(d_temp * dua,rc,vc),
        maxiter=50,
        )
    
    v_temp_ini=v_temp

    #print(v_temp/vc)

print('1 ',d_temp)
print('2 ',v_temp)
print('3 ', v_temp_ini)


In [ ]:
print(rc / dua)

In [ ]:
print(d /(rc / dua))
print(v_temp / vc)

In [ ]:
veff = sqrt((v_temp**2) + (vorb**2))
print(veff)


In [ ]:
sun = Star(
    name="Soleil",
    mass=1.0,
    radius={"models": star_radius_models, "radius": 1.0},
    age=AS,
    obs_dist=1.0,
    magfield=BSsw,
)
# sol.talk(talk=talk)
jup = Planet(
    name="Jupiter",
    mass=1.0,
    radius={"models": planet_radius_models, "radius": 1.0},
    distance=1.0,
    worb={"star_mass": MS, "worb": 1.0},
    wrot=1.0,
)